# -------------------- **1. Scraping a job sites with selenium** -------------------- #

In [ ]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import pandas as pd
import time

# set od options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Data frame construct
job_title_list = []
job_salary_list = []
job_place_list = []
job_name_company = []
res_comp = []
job_link_list = []
link_list = []
res =  []

options = Options()
ua = UserAgent()
a = ua.random
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')
url = 'https://www.topcv.vn/login'
driver = webdriver.Chrome(options=options, executable_path=r'C:\Users\Admin\Downloads\chromedriver.exe')
driver.get(url)

driver.maximize_window()

input_field_email = driver.find_element_by_xpath("//input[@placeholder='Email' and @name='email' and @class='form-control' and @type = 'email']").send_keys('lenhugiabinh@gmail.com')

input_field_pass = driver.find_element_by_xpath("//input[@placeholder='Mật khẩu' and @name='password' and @class='form-control' and @type = 'password']").send_keys('binhnhugiale123')

click_search = driver.find_element_by_xpath("//input[@type='submit']").click()

click_find = driver.find_element_by_xpath("//button[@class='form-control btn btn-block btn-topcv-primary btn-border btn-border-thin']").click()

for page in range(0,5):
    element = driver.find_element(By.CSS_SELECTOR, '[class="job-list search-result"]')
    html = element.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    # append title to job_title_list
    for job_title in soup.findAll('span',{ "class" : "bold transform-job-title"}):
            job_title_list.append(job_title.string)
    for job_salary in soup.findAll('span',{"class" : "text-highlight"}):
            job_salary_list.append(job_salary.string)
    # append the name of company to job_name_company
    for job_name in soup.findAll('a',{"data-toggle" : "tooltip"}):
        if job_name is not None:
            job_name_company.append(job_name.string)
    job_name_company = list(filter(None, job_name_company)) # delete None values in array
    # append the place to job_place_list
    for job_place in soup.findAll('span',{"class" : "address"}):
            job_place_list.append(job_place.string)
    # get link from title job
    for job_link in soup.findAll('a',{"target" : "_blank"}, href = True):
            job_link_list.append(job_link['href'])
    # append to link_list array link job
    for link in range(len(job_link_list)):
            if(link % 2 == 0):
                res.append(job_link_list[link])
    # remove duplicates values 
    for i in res:
        if i not in link_list:
            link_list.append(i)
    # Viết for loop ddeer lay them data
    element = driver.find_element_by_xpath("//a[@rel='next']")
    driver.execute_script("arguments[0].click();", element) 

# lấy thêm tên cty, khu vực, biến thành dạng dataframe
dict_to_df = {'Job title list': job_title_list, 'Salary list': job_salary_list, 'Name of the company':job_name_company}
df = pd.DataFrame(dict_to_df)
display(df)


In [10]:
display(df.head())

,Job title list,Salary list,Name of the company
0,Nhân Viên Điều Dưỡng Tại Phòng Tiêm Chủng Safpo/ Potec,Trên 6 triệu,Tập Đoàn Y Tế AMV GROUP
1,Lập Trình Viên Java,15-30 triệu,Công ty TNHH VNDCredit
2,Mobile Apps Developer (Android & IOS) - HN/ Lương Up To 30 Triệu,Tới 30 triệu,Công Ty TNHH MTV Viễn Thông Quốc Tế FPT
3,Frontend Developer,Thoả thuận,Công Ty CP The Mansions Việt Nam
4,Nhân Viên Marketing - Thái Nguyên,10-20 triệu,KORSY


# -------------------- **2. Scraping app store with request** -------------------- #

In [16]:
import json
import requests
import pandas as pd
import pandas_gbq
from datetime import date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from datetime import date, timedelta

url = 'https://api.appannie.com/v1.3/apps/google-play/app/########/reviews?start_date=2018-06-15&end_date=2018-08-15&page_size=10'
api_key = "######"
key = f'bearer {api_key}'
response = requests.get(url, headers={'Authorization':key})
data = response.json()
df_page = pd.DataFrame.from_dict(data['reviews'])
display(df_page.head())

,review_id,date,title,name,device,market_code,version,language_name,language,text,rating,country,reviewer
0,gp:AOqpTOGbR9MS7toiOyWO_BmZ7PAqxdL56z5KASmerZWFzeEltGBSFSjdeXVR_Hdea9F6wNNhqu06Ni-GKD1oGoU,2018-08-12,None,None,android-all,google-play,1.30.1 (6769),English,en,Please add in-app purchases like upgrading to premium membership through the app!,5,None,
1,gp:AOqpTOFYk_NXKcP3LDRlu3JM_i20W-MizWNe50HJgofg4vsVLAF7vRZaapjYKiZapHmfev0r00iBeC_tiS9T-Kc,2018-08-07,None,None,android-all,google-play,1.43.0 (8850),English,en,Unable to enter email address to register,1,None,
2,gp:AOqpTOF4-bi1F354KWbKJxfMl0oHesndk4Tcjhmf-M6gtPX0wGaCK7PdT7D6CBgKZXV5Y7mWAYg3oRz_huKbd1I,2018-07-27,None,None,android-all,google-play,1.42.0 (8846),English,en,Not being able to see version history of a specific app is a bit disappointing. You can see this when using the website instead.,3,None,
3,gp:AOqpTOGbR9MS7toiOyWO_BmZ7PAqxdL56z5KASmerZWFzeEltGBSFSjdeXVR_Hdea9F6wNNhqu06Ni-GKD1oGoU,2018-07-21,None,None,android-all,google-play,1.30.1 (6769),English,en,Please add in-app purchases like upgrading to premium membership through the app!,5,None,
4,gp:AOqpTOGUZ--Ey1DwU6iNWcwXalN_7BrF3IG4DE8_uSLcPWSbXtyggi4c4nBt6ul0uCBLd3W8FnqDk8dd3UyjaFA,2018-06-29,None,None,android-all,google-play,1.41.0 (8823),Japanese,ja,順位の低いゲームは載っていないのか見当たらない,1,None,
